## Why I use parquet for storing and then duckdb for analyis

- In production database systems (OLTP):
  - we mostly focus on rows not on columns --> row-based storing
  - we want to have highly normalized tables (lowers redundancy and improves integrity,)
  - we want transactions to be ACID
- In the Analytics  database systems (OLAP):
  - We might have huge amounts of data
  - we mostly focus on columns not rows --> column-based storing
  - we want to have highly denormalized we want to have highly denormalized data (1 combined table in Normalform 1 --> often has many attributes) or a snowflake structure, since this reduces joins, which are expensive.
  

Parquet, Duckdb and Pandas Dataframe are all column-based which is what we want, since this task is all about etl and analytics. DuckDB then gives us the advantage to use sql for complex data analysis queries (which would be more tricky with pandas). Furthermore, these 3 technologies work very well together, as they support export and import functionality for one another.

Note: Having just one table is the best approach for this use-case.


**what each tool is used for:**

- Pandas DataFrame = sanity checks, cleaning, transformations, quick stats
- DuckDB = complex SELECT statements for data-analysis tasks
- Parquet = ideal file format to persist large amounts of data

# Approach for this exercise

### Identifiying extraction targets in the underlying data

1. Currently we have html files as underlaying data
2. When we inspect the html files we can look for specifi html tags or  html tag attributes that enclose data which might be interesting to us.
3. What I found out by inspecting some files in vs-code:
   - there is only one ```<h1>``` tag per file
   - this ```<h1>``` tag is the first subtag within a certain div that does not have an id sadly
   - but inside this div are all the data we are intersted in
   - then there are some data-cy attributes in certain tags which futhermore give us an idea of content we might be interested in
   - so I just look at each data-cv attribute within this div to find extraction targets.
4. I find the following data-cv attrributes interesting:
   1. data-cy="vacancy-title"
   2. data-cy="info-pulbication" --> to find publishing date
   3. data-cy="info-workload" --> to find employment percentage
   4. data-cy="info-contract" --> to see if it is a permant position or else
   5. data-cy="info-language" --> to see required languages for this job
   6. data-cy="vacancy-description" --> get a detailed job description
   8. data-cy="company-industry"
   9. data-cy="company-employee-count"
   10. data-cy="company-link"
5. The problem with this data-cv however is that they are not present in every job posting html page and even if they are, not every job posting has the the same data-cv values present.
6. For this reason I'm goign the use a different way to extract data I'm interested in.
7. Most pages have all the interesting data in a json format presence that looks like this:
  
```
<script type="application/ld+json">
      [
        {
          "@context": "http://schema.org/",
          "@type": "JobPosting",
          "title": "Teamleitung Operations (80-100%)",
          "description": "<p></p><p>Unsere Mandantin ist eine etablierte Schweizer IT-Dienstleisterin mit Fokus auf IT-Outsourcing, Microsoft-Technologien, Modern Workplace und Cloud-Lösungen. Seit über 25 Jahren unterstützt das Unternehmen KMU in der Schweiz dabei, eine kontinuierlich verfügbare und sichere IT-Infrastruktur zu gewährleisten. Die Unternehmenskultur ist geprägt von Kundennähe, Stabilität und einem hohen Qualitätsanspruch.</p> <p>Im Zuge einer Nachfolgeregelung suchen wir den Kontakt zu einer führungserfahrenen Persönlichkeit, die gewohnt ist, in einem turbulenten IT-Alltag die Ruhe zu bewahren und die richtigen Parameter zu setzen.</p><p></p><p></p><p> <strong>Ihre Schwerpunkte:</strong> </p> <ul> <li>Sicherstellen einer hohen Kundenzufriedenheit durch zuverlässige Services und partnerschaftliche Zusammenarbeit</li> <li>Gewährleisten der Servicequalität sowie Einhaltung von SLAs im laufenden Betrieb</li> <li>Enge Zusammenarbeit mit Kunden, Partnern und internen Fachbereichen</li> <li>Aktive Mitwirkung bei der Weiterentwicklung des Serviceportfolios sowie Unterstützung in Pre-Sales-Aktivitäten</li> <li>Führen, Fördern und Weiterentwickeln des Operations-Teams</li> <li>Verantworten der Ressourcenplanung und des Personaleinsatzes</li> </ul> <p> <strong>Was bringen Sie mit?</strong> </p> <ul> <li>Ausgeprägte Kunden- und Serviceorientierung</li> <li>Fähigkeit, Mitarbeitende zu inspirieren und weiterzuentwickeln</li> <li>Mehrjährige Führungserfahrung in einer vergleichbaren Rolle, basierend auf einem fundierten technischen Hintergrund in der Informatik und entsprechenden Weiterbildungen</li> <li>Umfassende Erfahrung mit Schwerpunkt Betrieb in Microsoft-Technologien – insbesondere Azure und Microsoft 365 – sowie Hybrid- und Cloud-Umgebungen; Erfahrung mit Citrix von Vorteil</li> <li>Praxis im IT-Outsourcing und Betrieb von Kundeninfrastrukturen – verbunden mit der Fähigkeit, sowohl kurzfristig souverän zu handeln als auch langfristig Kundenbeziehungen und Performance sicherzustellen</li> <li>Freude an der Gestaltung und Weiterentwicklung von Services sowie Bereitschaft, die Unternehmenskultur aktiv mitzuprägen</li> </ul> <p> <strong>Das Angebot:</strong> </p> <ul> <li>Eine verantwortungsvolle Schlüsselrolle mit Gestaltungsspielraum</li> <li>Ein motiviertes, kompetentes Team und ein wertschätzendes Arbeitsumfeld</li> <li>Moderne Arbeitsmodelle, Weiterbildungs- und Entwicklungsmöglichkeiten</li> <li>Ein Unternehmen, das konsequent auf Qualität, Stabilität und Kundennähe setzt</li> </ul> <p>Auf Sie wartet ein tolles Führungsteam, engagierte Mitarbeitende und kein 0815-Job. Ihr neuer Arbeitsort ist in der Agglomeration Zürich. Nun liegt es ganz an Ihnen, sich für diese fantastische Teamleitungsfunktion in diesem KMU zu bewerben. Frau Monica Fischer freut sich auf Ihre kompletten Bewerbungsunterlagen. Zögern Sie nicht, mich bei Fragen zu kontaktieren. Ich bin gerne für Sie da!</p><p></p><p></p>",
          "datePosted": "2025-09-18T12:36:34+02:00",
          "hiringOrganization": {
            "@type": "Organization",
            "name": "CUBUS Employability Competence Center GmbH",
            "sameAs": "https://www.cubus-employability.ch/",
            "logo": "https://media.jobs.ch/images/e2369571-52a5-4f40-89f5-e2f0a88cd745/4014x890.png"
          },
          "jobLocation": {
            "@type": "Place",
            "address": {
              "@type": "PostalAddress",
              "streetAddress": "",
              "addressLocality": "Grossraum Zürich",
              "addressCountry": "Switzerland"
            }
          },
          "industry": "Informatik / Telekommunikation",
          "employmentType": ["PART_TIME", "FULL_TIME"],
          "occupationalCategory": "Beratung / Wirtschaftsinformatik"
        }
      ]
    </script>
```



#### Choosen Extraction Targets (json-style)
- **data.title** -> good for filtering and getting a first Idea of the Job
- **data.description** --> for text analysis and futher filtering
- **data.datePosted** --> to see when the job was posted
- **data.hiringOrganization.name** --> name of hiring company
- **data.hiringOrganization.sameAs** --> link of hring company
- **data.jobLocation.address.addressLocality** --> to see the region of the job
- **data.employmentType** --> to see if they offer part_time, full_time or both
- **data.occupationalCategory** --> to further filter the category within an industry









In [1]:
%pip install bs4

^C
Note: you may need to restart the kernel to use updated packages.


# Extract the Data targets
1. Extract as rows
2. Return all the rows
3. Transform into a Dataframe
4. Inspect the Dataframe
5. From the Dataframe we can save them in different formats
6. I decide to store them in parquet since:
   1.  it offers better compression than csv 
   2.  when we store it in parquet and read it via pandas or duckDB we can read direclty only the columns we interested in --> much faster
   3.  data types are preserved
   4.  DuckDB natively reads parquet. example `SELECT * FROM "jobs.parquet"`

In [ ]:
import json, glob, pathlib
from bs4 import BeautifulSoup
import pandas as pd

def iter_jobpostings_from_soup(soup):
    for tag in soup.find_all("script", type="application/ld+json"):
        raw = tag.string or ""
        try:
            data = json.loads(raw)
        except Exception:
            continue  # skip malformed blocks 

        # normalize to a flat list of candidate dicts
        candidates = []
        if isinstance(data, dict):
            candidates.append(data)
            # check @graph if present
            if isinstance(data.get("@graph"), list):
                candidates.extend(data["@graph"])
        elif isinstance(data, list):
            candidates.extend(data)

        # yield only JobPosting objects
        for obj in candidates:
            if isinstance(obj, dict) and obj.get("@type") == "JobPosting":
                yield obj

def collect_records(html_dir):
    rows = []
    limit = 2000
    counter = 0

    for path in glob.glob(str(pathlib.Path(html_dir) / "*.html")):
        if counter >= limit:
            break
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            soup = BeautifulSoup(f.read(), "lxml")

        for jp in iter_jobpostings_from_soup(soup):
            if counter >= limit:
                break  # stop if reached 1400
            rows.append({
                "file": pathlib.Path(path).name,
                "title": jp.get("title"),
                "description": jp.get("description"),  # keep HTML as-is for simplicity
                "datePosted": jp.get("datePosted"),
                "companyName": jp.get("hiringOrganization", {}).get("name"),
                "companyLink": jp.get("hiringOrganization", {}).get("sameAs"),
                "jobLocation": jp.get("jobLocation", {}).get("address", {}).get("addressLocality"),
                "employmentType": jp.get("employmentType"),
                "occupationalCategory": jp.get("occupationalCategory"),
            })
            counter += 1  

    return rows


# main code
html_folder = "./scraped_jobs_2025_10_15___07_46_29"   # change to your folder with the 1000+ files if name differs from mine
df = pd.DataFrame(collect_records(html_folder))
# after building your DataFrame you can sort out or clean data as needed
df = df.fillna("unknown")




In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   file                  2000 non-null   object
 1   title                 2000 non-null   object
 2   description           2000 non-null   object
 3   datePosted            2000 non-null   object
 4   companyName           2000 non-null   object
 5   companyLink           2000 non-null   object
 6   jobLocation           2000 non-null   object
 7   employmentType        2000 non-null   object
 8   occupationalCategory  2000 non-null   object
dtypes: object(9)
memory usage: 140.8+ KB


In [ ]:
df.head(10)[["title", "description"]]


,title,description
0,Enterprise IT Technologie Architekt 80-100 % (...,<p></p><p></p><p> <b>Das kannst du bei uns bew...
1,Professional System Engineer,<p></p><p><strong>Unternehmensbeschreibung</st...
2,Junior Software-Entwickler(a) 60 % bis 100%,<p></p><p></p><div> <div> <h2>Herzlich willkom...
3,ServiceDesk Engineer,<p></p><p><strong>Unternehmensbeschreibung</st...
4,IT Supporter:in (a) 60-100% [Ref:2189],"<p></p><p style=""margin:0cm;"" class=""Text"">Fre..."
5,Senior VoIP Support Engineer (a) 80-100%,<p>Du hast Erfahrung im Support und bist fit a...
6,Senior System Engineer,<p></p><p><strong>Unternehmensbeschreibung</st...
7,SAP HCM Consultant (m/w/d),<p></p><p></p><div> <p>Du interessierst dich d...
8,Software Engineer LIMS,<p></p><p>Die B. Braun Medical AG ist eine Toc...
9,Informatiker:in Applikationsentwicklung EFZ 100%,"<p></p><p></p><div> <div> <div class=""job""> <b..."


In [ ]:
df.tail(10)

,file,title,description,datePosted,companyName,companyLink,jobLocation,employmentType,occupationalCategory
1990,stellenausschreibung_963.html,Senior Project Leader 80 - 100 % (w/m/d),"<p></p><p></p><div>Getreu der Vision ""We drive...",2025-09-23T03:00:00+02:00,Hamilton Bonaduz AG,https://jobs.hamilton.ch/,Bonaduz,"[PART_TIME, FULL_TIME]",Projekt Management / Analyse
1991,stellenausschreibung_964.html,Service Specialist - Rollout 60 - 70% (m/w/d),<p></p><p>Factory 77 AG - Wir gestalten die Zu...,2025-09-23T03:00:00+02:00,Factory Seventy Seven,https://factory77.ch/,Hochdorf,[PART_TIME],User Help Desk / Support / Training
1992,stellenausschreibung_965.html,Senior Application Engineer (80-100%),<p></p><p>SIX treibt den Wandel der Finanzmärk...,2025-09-23T03:00:00+02:00,SIX,https://www.six-group.com/en/careers.html,Zurich,"[PART_TIME, FULL_TIME]",Software Programmierung
1993,stellenausschreibung_966.html,System Engineer (m/w/d),<p></p><p></p><div> <div> <div> <p>System Engi...,2025-09-22T19:32:00+02:00,EMS-CHEMIE AG,https://www.ems-group.com,Domat/Ems,[FULL_TIME],System Engineering
1994,stellenausschreibung_967.html,Account Manager (w/m/d),<p></p><div> <p>Bereit für alles? Bereit für B...,2025-09-22T18:35:47+02:00,Bechtle Direct AG Rotkreuz,https://www.bechtle.com,Rotkreuz,[FULL_TIME],System Administration
1995,stellenausschreibung_968.html,Global Logistics & Customization Project Manag...,"<p></p><p>At Belimo, we take pride in making o...",2025-09-22T18:34:33+02:00,BELIMO Automation AG,https://www.belimo.ch/karriere,Hinwil,"[PART_TIME, FULL_TIME]",ERP / SAP / CRM
1996,stellenausschreibung_969.html,Electrical / Software Engineer 80 - 100 % (w/m/d),"<p></p><p></p><div>Getreu der Vision ""We drive...",2025-09-22T17:35:29+02:00,Hamilton Bonaduz AG,https://jobs.hamilton.ch/,Rapperswil,"[PART_TIME, FULL_TIME]",Software Architektur / Engineering
1997,stellenausschreibung_97.html,"FinOps Service Manager (Azure), 80-100% (w/m/d)",<p>Als FinOps Service Manager (Azure) bist du ...,2025-10-06T05:04:30+02:00,Swiss Life AG,https://www.swisslife.ch/de/ueber-uns/karriere...,Zürich,"[PART_TIME, FULL_TIME]",Software Architektur / Engineering
1998,stellenausschreibung_970.html,Senior Cyber Security Analyst & Incident Respo...,<p></p><p>Tagtäglich wird in den Medien über C...,2025-09-22T17:34:38+02:00,AVANTEC AG,https://www.avantec.ch,Zürich,"[PART_TIME, FULL_TIME]",Testing / Audit / Security
1999,stellenausschreibung_971.html,Microsoft Dynamics 365 HR Technical Consultant...,<p></p><p>HR Campus macht HR zum Erfolgsfaktor...,2025-09-22T17:05:12+02:00,HR Campus AG,https://www.hr-campus.ch/de/,Dübendorf,"[PART_TIME, FULL_TIME]",Beratung / Unternehmensentwicklung


In [ ]:
df.sample(20)

,file,title,description,datePosted,companyName,companyLink,jobLocation,employmentType,occupationalCategory
420,stellenausschreibung_1386.html,IT System Supporter (m/w/d),"<div class=""info-block-content"">\n<p>Rocken® i...",2025-10-14T12:32:06+02:00,Rocken®,https://www.jobs.ch/de/firmen/105817-rocken/,Frauenfeld,"[PART_TIME, FULL_TIME]",System Administration
1410,stellenausschreibung_439.html,Full-Stack Engineer Business Applications,<p>Gestalte mit uns die Zukunft des E-Commerce...,2025-09-26T04:04:48+02:00,Brack.Alltron,https://brackalltron.ch/,Mägenwil,"[PART_TIME, FULL_TIME]",Software Programmierung
658,stellenausschreibung_1624.html,Stadler: Cyber Security Engineer (PSOC),"<div class=""info-block-content"">\n<p>Stadler b...",2025-10-09T11:13:16+02:00,Stadler Rail Group,https://www.jobs.ch/de/firmen/20190-stadler-ra...,Bussnang,[FULL_TIME],Testing / Audit / Security
996,stellenausschreibung_1930.html,Senior Software Engineer (Angular/Java),"<div class=""info-block-content"">\n<p>Entwickle...",2025-09-26T10:23:18+02:00,V-ZUG AG,https://www.jobs.ch/de/firmen/3545-v-zug-ag/,Zug,[FULL_TIME],Software Architektur / Engineering
1901,stellenausschreibung_883.html,Avaloq Developer Payment oder Securities 80-10...,<p></p><p></p><div> <div> <h2>Über uns</h2> </...,2025-09-25T16:04:26+02:00,LGT,https://www.lgt.com,Lugano,"[PART_TIME, FULL_TIME]",Fonds / Wertschriften / Handel
100,stellenausschreibung_1089.html,Senior IT System Engineer (m/w/d) 100%,<p></p><p> <strong>Willkommen bei Elis – Dein ...,2025-09-17T16:04:17+02:00,Elis (Suisse) AG,https://www.elis.com,Bern,[FULL_TIME],System Engineering
1681,stellenausschreibung_683.html,"Business Developer (m/w) – IT-Security, IoT & ...",<p></p><p></p><p> </p> <p>Zur Verstärkung unse...,2025-10-02T13:18:45+02:00,Inalp Solutions AG,https://inalp.com/,Niederwangen BE,"[PART_TIME, FULL_TIME]",Software Architektur / Engineering
1323,stellenausschreibung_360.html,Projektleiter:in Public,<p></p><h3>DU BIST DER #HAMMER</h3> <p>Wir suc...,2025-10-10T11:35:09+02:00,AKROS AG,https://www.akros.ch/,"Bern, Biel, Zürich oder Luzern","[PART_TIME, FULL_TIME]",Beratung / Wirtschaftsinformatik
1749,stellenausschreibung_745.html,Senior Netzwerk Security Engineer,<p></p><div>Im Auftrag der axelion AG suchen w...,2025-10-01T03:09:02+02:00,Avalect Personaldienstleistungen,https://www.avalect.ch,Baden und Rothenburg,[FULL_TIME],Testing / Audit / Security
1405,stellenausschreibung_434.html,Junior Projektmitarbeiter Digitalisierung (m/w),"<p></p><p>Ob Student, Berufseinsteiger oder be...",2025-10-09T13:35:56+02:00,univativ Schweiz AG,https://www.univativ.ch,Luzern/LU,"[PART_TIME, FULL_TIME]",Beratung / Wirtschaftsinformatik


In [ ]:
# save as jobs.parquet (Best format for DuckDB)
df.to_parquet("jobs.parquet", index=False)   # preferred
# Optional: also a CSV if you want
df.to_csv("jobs.csv", index=False)
print(df.shape, "rows written")


(2000, 9) rows written


In [ ]:
#comparing the files sizes parquet vs csv to see if parquet is smaller
import os
parquet_size = os.path.getsize("jobs.parquet")
csv_size = os.path.getsize("jobs.csv")
print(f"Parquet size: {parquet_size} bytes")
print(f"CSV size: {csv_size} bytes")
print(f"Parquet compression rate: {csv_size / parquet_size:.2f}")

Parquet size: 3757773 bytes
CSV size: 7959848 bytes
Parquet compression rate: 2.12


# Loading Step (using duckdb)

In [2]:
import duckdb

# Create or replace the 'jobs' table and load data from Parquet
duckdb.sql("""
CREATE OR REPLACE TABLE jobs AS
SELECT
  row_number() OVER () AS id,  -- creates sequential IDs starting at 1
  cast(file AS VARCHAR) AS file,
  cast(title AS VARCHAR) AS title,
  cast(description AS VARCHAR) AS description,
  CAST(datePosted AS TIMESTAMPTZ) AS datePosted,
  cast("companyName" AS VARCHAR) AS companyName,
  cast("companyLink" AS VARCHAR) AS companyLink,
  cast(jobLocation AS VARCHAR) AS jobLocation,
  cast(employmentType AS VARCHAR) AS employmentType,
  cast(occupationalCategory AS VARCHAR) AS occupationalCategory
FROM 'jobs.parquet';
""")
# Show the schema of the 'jobs' table
duckdb.sql("DESCRIBE jobs").show()


┌──────────────────────┬──────────────────────────┬─────────┬─────────┬─────────┬─────────┐
│     column_name      │       column_type        │  null   │   key   │ default │  extra  │
│       varchar        │         varchar          │ varchar │ varchar │ varchar │ varchar │
├──────────────────────┼──────────────────────────┼─────────┼─────────┼─────────┼─────────┤
│ id                   │ BIGINT                   │ YES     │ NULL    │ NULL    │ NULL    │
│ file                 │ VARCHAR                  │ YES     │ NULL    │ NULL    │ NULL    │
│ title                │ VARCHAR                  │ YES     │ NULL    │ NULL    │ NULL    │
│ description          │ VARCHAR                  │ YES     │ NULL    │ NULL    │ NULL    │
│ datePosted           │ TIMESTAMP WITH TIME ZONE │ YES     │ NULL    │ NULL    │ NULL    │
│ companyName          │ VARCHAR                  │ YES     │ NULL    │ NULL    │ NULL    │
│ companyLink          │ VARCHAR                  │ YES     │ NULL    │ NULL    

In [3]:
duckdb.sql("SELECT COUNT(*) AS total_jobs FROM jobs").show()

┌────────────┐
│ total_jobs │
│   int64    │
├────────────┤
│       2000 │
└────────────┘



In [4]:
# Query example: Get the 10 most recent job postings not from 'Rocken' in specified locations
duckdb.sql("""SELECT * FROM JOBS  
           WHERE CompanyName NOT LIKE '%Rocken%' and jobLocation LIKE '%Bern%' OR jobLocation LIKE '%Biel%' OR JobLocation LIKE '%THUN%' OR JobLocation LIKE '%Lyss%' OR JobLocation LIKE '%BRÜGG%' OR JobLocation LIKE '%Solothurn%'
           ORDER BY datePosted DESC LIMIT 10""").show()

┌───────┬────────────────────────────────┬──────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

### Save the data
We now have structured data that also has the right data-types. We even some test select statements.
As a next step I want to save this strucutre data for example again to parquet file.

In [6]:
duckdb.sql("""
    SELECT * FROM JOBS
""").to_df().to_parquet("structured_jobs.parquet", index=False)  